In [21]:
import pandas as pd

# Load the dataset to check its structure
data_path = "weather_data_2013_to_2023_cleaned.csv"
data = pd.read_csv(data_path)
data.head()


,date,time,Cloud Coverage (%),Temperature (Fahrenheit),Wind speed (mph),rain (inches),relative humidity (%),visibility (miles)
0,2013-01-01,00:00:00,0,36.86,5.82,0.0,83,0.0
1,2013-01-01,01:00:00,1,37.04,3.36,0.0,83,0.0
2,2013-01-01,02:00:00,4,36.32,4.70,0.0,83,0.0
3,2013-01-01,03:00:00,1,36.68,3.36,0.0,83,0.0
4,2013-01-01,04:00:00,20,36.14,2.24,0.0,85,0.0


In [22]:
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np

data = pd.read_csv(data_path)
data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'])
data.set_index('datetime', inplace=True)
data.drop(['date', 'time'], axis=1, inplace=True)

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)
n_input = 6
n_features = data_scaled.shape[1]  
generator = TimeseriesGenerator(data_scaled, data_scaled, length=n_input, batch_size=1)
generator[0][0].shape, generator[0][1].shape



((1, 6, 6), (1, 6))

In [23]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

generator = TimeseriesGenerator(data_scaled, data_scaled, length=n_input, batch_size=1)
generator[0][0].shape, generator[0][1].shape




((1, 6, 6), (1, 6))

In [24]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(100, activation='relu', input_shape=(24, n_features)),  # 24 time steps
    tf.keras.layers.Dense(n_features)
])

model.compile(optimizer='adam', loss='mse')
model.fit(generator, epochs=2)



Epoch 1/2
22999/22999 [==============================] - 42s 2ms/step - loss: 0.0095
Epoch 2/2
22999/22999 [==============================] - 41s 2ms/step - loss: 0.0081


In [25]:

test_input = data_scaled[-n_input:]  # Last available input sequence
current_batch = test_input.reshape((1, n_input, n_features))
predicted = model.predict(current_batch)[0]
predicted


ValueError: in user code:

    File "/Users/ral/Library/Python/3.9/lib/python/site-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/Users/ral/Library/Python/3.9/lib/python/site-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/ral/Library/Python/3.9/lib/python/site-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/ral/Library/Python/3.9/lib/python/site-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/Users/ral/Library/Python/3.9/lib/python/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/ral/Library/Python/3.9/lib/python/site-packages/keras/src/engine/input_

In [ ]:
input_seq = test_input.reshape((1, n_input, n_features))

predicted_hours = []
for _ in range(20):
    next_hour_pred = model.predict(input_seq)[0]
    predicted_hours.append(next_hour_pred)

    input_seq = np.append(input_seq[:, 1:, :], [next_hour_pred], axis=1)

predicted_hours_actual = scaler.inverse_transform(predicted_hours)


1/1 [==============================] - 0s 12ms/step


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)